### Optical Flow 

- Import esential libraries 

In [24]:
import numpy as np 
import cv2 as cv 

In [27]:
corner_track_params = dict(maxCorners=15,qualityLevel=0.3,minDistance=7,blockSize=7)
lk_params = dict(winSize=(200,200),maxLevel=2,criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,10,0.03))

The `corner_track_params` dictionary contains parameters for the cv.goodFeaturesToTrack function, which is used to detect good features (corners) to track in an image. The parameters are:

- maxCorners=10: This specifies the maximum number of corners to return. If there are more corners than this, only the strongest ones are returned.
- qualityLevel=0.3: This parameter characterizes the minimal accepted quality of image corners. It is a value between 0 and 1, where a higher value means only the best corners are selected.
- minDistance=7: This is the minimum possible Euclidean distance between the returned corners. It ensures that the detected corners are spaced out by at least this distance.
- blockSize=7: This size of the block used for computing the covariance matrix of derivatives at each pixel neighborhood.

The `lk_params` dictionary contains parameters for the cv.calcOpticalFlowPyrLK function, which implements the Lucas-Kanade method for calculating optical flow. 

The parameters are:
- winSize=(200,200): This specifies the size of the search window at each pyramid level. A larger window size can capture larger movements but may be slower and less accurate for small movements.
- maxLevel=2: This indicates the number of pyramid levels, including the initial image. A higher number allows for capturing larger movements but increases computational complexity.
- criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,10,0.03): This is the termination criteria of the iterative search algorithm. It can terminate either after a certain number of iterations (cv.TERM_CRITERIA_COUNT) or when the search window moves by less than a certain amount (cv.TERM_CRITERIA_EPS). Here, it will stop after 10 iterations or when the search window moves by less than 0.03 pixels.

In [29]:
cap = cv.VideoCapture(0)
ret,previous_frame = cap.read()

previous_gray = cv.cvtColor(src=previous_frame,code=cv.COLOR_BGR2GRAY)

# ASSIGN POINTS TO TRACK
previous_points = cv.goodFeaturesToTrack(image=previous_gray,mask=None,**corner_track_params)

drawing_mask = np.zeros_like(previous_frame)

while True:
  ret,current_frame = cap.read()
  current_gray = cv.cvtColor(current_frame,cv.COLOR_BGR2GRAY)

  next_points, status,error = cv.calcOpticalFlowPyrLK(prevImg=previous_gray,nextImg=current_gray,prevPts=previous_points,nextPts=None,**lk_params)

  current_tracked_features = next_points[status == 1]
  previous_tracked_features = previous_points[status == 1]

  for i, (new, old) in enumerate(zip(current_tracked_features, previous_tracked_features)):
      x_new, y_new = new.ravel()
      x_old, y_old = old.ravel()
      pt1 = (int(x_new), int(y_new))
      pt2 = (int(x_old), int(y_old))
      drawing_mask = cv.line(img=drawing_mask, pt1=pt1, pt2=pt2, color=(0, 255, 0), thickness=2)
      current_frame = cv.circle(current_frame, center=pt1, radius=5, color=(0, 0, 255), thickness=-1)

  img = cv.add(current_frame, drawing_mask)
  cv.imshow('Track', img)

  k = cv.waitKey(30) & 0xFF
  if k == 27:
      break
  
  previous_gray = current_gray.copy()
  previous_points = current_tracked_features.reshape(-1, 1, 2)


cap.release()
cv.destroyAllWindows()

### Step-by-Step Explanation

1. **Start the Loop**:
   - `while True:` starts an infinite loop that will keep running until we explicitly break out of it.

2. **Capture a Frame**:
   - `ret, current_frame = cap.read()`: This line captures a frame from the video feed. `cap.read()` returns two values: `ret` (a boolean indicating if the frame was captured successfully) and `current_frame` (the captured frame).

3. **Convert to Grayscale**:
   - `current_gray = cv.cvtColor(current_frame, cv.COLOR_BGR2GRAY)`: This line converts the captured frame to a grayscale image. Optical flow algorithms work better with grayscale images.

4. **Calculate Optical Flow**:
   - `next_points, status, error = cv.calcOpticalFlowPyrLK(prevImg=previous_gray, nextImg=current_gray, prevPts=previous_points, nextPts=None, **lk_params)`: This line calculates the optical flow using the Lucas-Kanade method. It tracks the movement of points from the previous frame (`previous_gray`) to the current frame (`current_gray`). It returns:
     - `next_points`: The new positions of the tracked points.
     - `status`: An array indicating whether the points were successfully tracked.
     - `error`: An array of errors for the tracked points.

5. **Filter Tracked Points**:
   - `current_tracked_features = next_points[status == 1]`: This line keeps only the points that were successfully tracked in the current frame.
   - `previous_tracked_features = previous_points[status == 1]`: This line keeps only the points that were successfully tracked in the previous frame.

6. **Draw Lines and Circles**:
   - `for i, (new, old) in enumerate(zip(current_tracked_features, previous_tracked_features)):`: This loop iterates over pairs of new and old points.
     - `x_new, y_new = new.ravel()`: Extracts the x and y coordinates of the new point.
     - `x_old, y_old = old.ravel()`: Extracts the x and y coordinates of the old point.
     - `pt1 = (int(x_new), int(y_new))`: Creates a tuple for the new point.
     - `pt2 = (int(x_old), int(y_old))`: Creates a tuple for the old point.
     - `drawing_mask = cv.line(img=drawing_mask, pt1=pt1, pt2=pt2, color=(0, 255, 0), thickness=2)`: Draws a green line on the `drawing_mask` from the old point to the new point.
     - `current_frame = cv.circle(current_frame, center=pt1, radius=5, color=(0, 0, 255), thickness=-1)`: Draws a red circle on the current frame at the new point.

7. **Combine and Display the Image**:
   - `img = cv.add(current_frame, drawing_mask)`: Combines the current frame and the drawing mask to create the final image.
   - 

cv.imshow('Track', img)

: Displays the final image in a window named 'Track'.

8. **Check for Exit Key**:
   - `k = cv.waitKey(30) & 0xFF`: Waits for 30 milliseconds for a key press. The `& 0xFF` ensures we get the correct key code.
   - `if k == 27:`: Checks if the 'Esc' key (ASCII code 27) was pressed. If so, it breaks out of the loop.

9. **Update Previous Frame and Points**:
   - `previous_gray = current_gray.copy()`: Updates the previous frame to be the current frame.
   - `previous_points = current_tracked_features.reshape(-1, 1, 2)`: Updates the previous points to be the current tracked points.

10. **Release Resources**:
    - `cap.release()`: Releases the video capture object.
    - 

cv.destroyAllWindows()

: Closes all OpenCV windows.

This loop continuously captures frames from the video, tracks the movement of points, draws lines and circles to show the movement, and displays the result until the 'Esc' key is pressed.